In [12]:
import pandas as pd
import numpy as np

In [15]:
checkpoint = pd.read_csv('./clean_data/checkpoint_cols.csv')
svi = pd.read_csv('./clean_data/svi_clean.csv')
vaccine = pd.read_csv('./clean_data/vaccine_hesitancy.csv')
election = pd.read_csv('./clean_data/final_clean_vote_data.csv')
census = pd.read_csv('./clean_data/census_data.csv')

In [16]:
election

,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden
0,DE,Joe Biden,DEM,44552,87025,0.4880,0.5120
1,DE,Joe Biden,DEM,195034,287633,0.3220,0.6780
2,DC,Joe Biden,DEM,39041,41681,0.0630,0.9370
3,FL,Joe Biden,DEM,89704,142604,0.3710,0.6290
4,FL,Joe Biden,DEM,618752,957672,0.3540,0.6460
...,...,...,...,...,...,...,...
3136,MA,Joe Biden,DEM,9914,12873,0.2044,0.7704
3137,MA,Joe Biden,DEM,63362,88332,0.2522,0.7173
3138,MA,Joe Biden,DEM,617196,869347,0.2611,0.7100
3139,MA,Joe Biden,DEM,270522,337240,0.1738,0.8022


In [46]:
#census.isnull().sum()
#went through columns with nans; the mismatched indices are not due to nans
#census[census['Census2019_18to64Pop'].isnull()]

In [47]:
#rename vaccine and election to match County_State columns
vaccine.rename(columns={'county_state':'County_State'}, inplace=True)
election.rename(columns={'county_state':'County_State'}, inplace=True)

In [48]:
svi.rename(columns={'index':'County_State'}, inplace=True)

In [49]:
print(census[census['County_State']=='aleutians east borough_ak'].loc[649,'County_State'])
print(vaccine[vaccine['County_State']=='aleutians east borough_ak'].loc[32,'County_State'])
print(checkpoint[checkpoint['County_State']=='aleutians east borough_ak'].loc[649,'County_State'])

print((census[census['County_State']=='aleutians east borough_ak'].loc[649,'County_State']) == (vaccine[vaccine['County_State']=='aleutians east borough_ak'].loc[32,'County_State']))
print((vaccine[vaccine['County_State']=='aleutians east borough_ak'].loc[32,'County_State']) == (checkpoint[checkpoint['County_State']=='aleutians east borough_ak'].loc[649,'County_State']))

aleutians east borough_ak
aleutians east borough_ak
aleutians east borough_ak
True
True


In [50]:
#add new county columns to double check they merge correctly. call them all diff name
checkpoint['county_checkpoint']=checkpoint['County_State']
svi['county_svi']=svi['County_State']
vaccine['county_vaccine']=vaccine['County_State']
election['county_election']=election['County_State']
census['county_census']=census['County_State']

In [51]:
#create list of each df's county_state's
checkpoint_county_list = list(checkpoint['County_State'])
svi_county_list = list(svi['County_State'])
vaccine_county_list = list(vaccine['County_State'])
election_county_list = list(election['County_State'])
census_county_list = list(census['County_State'])

lists = [checkpoint_county_list, svi_county_list, vaccine_county_list,
        election_county_list, census_county_list]

In [52]:
#lengths of lists
#
def length(lst, name):
    print(f'{name} length: {len(lst)}')

length(checkpoint_county_list, 'checkpoint')
length(svi_county_list, 'svi')
length(vaccine_county_list, 'vaccine')
length(election_county_list, 'election')
length(census_county_list, 'census')

checkpoint length: 3191
svi length: 3142
vaccine length: 3142
election length: 4627
census length: 3191


In [53]:
#print shapes
dfs = [checkpoint,svi,vaccine,election,census]

def shapes(df, dfname):
    print(f'{dfname} shape: {df.shape}')
    
def total_shape(df_list):
    count = 0
    for i in df_list:
        count += i.shape[1]
    print(f'total columns={count}')

In [54]:
shapes(checkpoint,'checkpoint')
shapes(svi, 'svi')
shapes(vaccine, 'vaccine')
shapes(election, 'election')
shapes(census, 'census')
total_shape(dfs)

checkpoint shape: (3191, 14)
svi shape: (3142, 15)
vaccine shape: (3142, 6)
election shape: (4627, 10)
census shape: (3191, 9)
total columns=54


In [55]:
#set indices to 'County_State'
#checked and they're all indexed correctly
for i in dfs:
    i.set_index('County_State', inplace=True)

In [56]:
#merge: checkpoint,vaccine,election,census
#no svi
df1_nosvi = pd.merge(checkpoint,census, left_index=True, right_index=True)
df2_nosvi = pd.merge(vaccine,election, left_index=True, right_index=True)
df_nosvi = pd.merge(df1_nosvi,df2_nosvi, left_index=True, right_index=True)

In [57]:
#merge: checkpoint,svi,vaccine,election,census
df1 = pd.merge(checkpoint,svi, left_index=True, right_index=True)
df2 = pd.merge(vaccine,election, left_index=True, right_index=True)
df3 = pd.merge(census,df1, left_index=True, right_index=True)
df = pd.merge(df2,df3, left_index=True, right_index=True)

In [58]:
print(f'df_nosvi shape: {df_nosvi.shape}')
print(f'df_nosvi shape: {df.shape}')

df_nosvi shape: (3088, 35)
df_nosvi shape: (3088, 49)


In [59]:
#df.to_csv('./clean_data/merged.csv')
#df_nosvi.to_csv('./clean_data/merged_nosvi.csv')

In [17]:
#make sure indices worked by checking with the extra county_state columns I added to the original dataframes
df[['county_vaccine','county_election', 'county_checkpoint','county_svi','county_census']]
print((df['county_vaccine']==df['county_election']).mean())
print((df['county_election']==df['county_checkpoint']).mean())
print((df['county_checkpoint']==df['county_svi']).mean())
print((df['county_svi']==df['county_census']).mean())

1.0
1.0
1.0
1.0


In [18]:
#print(difference(df_county_list,svi_county_list))
#print(difference(df_county_list,vaccine_county_list))
#print(difference(df_county_list,election_county_list))
#print(difference(df_county_list,census_county_list))

In [19]:
#see differences in lists of county_states
#merged df without svi
df_nosvi_county_list = list(df_nosvi['county_vaccine'])

def difference(lst):
    final_list = list(set(lst) - set(df_nosvi_county_list))
    return final_list

checkpoint_diff = difference(checkpoint_county_list)
vaccine_diff = difference(vaccine_county_list)
election_diff = difference(election_county_list)
census_diff = difference(census_county_list)

In [20]:
print(f'df_nosvi checkpoint diff: {len(checkpoint_diff)}')
print(f'df_nosvi vaccine diff: {len(vaccine_diff)}')
print(f'df_nosvi election diff: {len(election_diff)}')
print(f'df_nosvi svi census: {len(census_diff)}')

df_nosvi checkpoint diff: 103
df_nosvi vaccine diff: 54
df_nosvi election diff: 1539
df_nosvi svi census: 103


In [21]:
#add 49 empty entries to svi_diff and vaccine_diff so can add everything to a df
#vacc_append = ['ZZZZ']*1485
#check_census_append = ['ZZZZ']*1436
#vaccine_diff = vaccine_diff+vacc_append
#checkpoint_diff = checkpoint_diff+check_census_append
#census_diff = census_diff+check_census_append

In [22]:
#diffs_nosvi = pd.DataFrame()
#diffs_nosvi['checkpoint_diff']=checkpoint_diff
#diffs_nosvi['vaccine_diff']=vaccine_diff
#diffs_nosvi['election_diff']=election_diff
#diffs_nosvi['census_diff']=census_diff
#diffs_nosvi.to_csv('./clean_data/diffs_nosvi.csv')

In [23]:
################### repeat with merged df including svi

In [24]:
#see differences in lists of county_states
#merged df with svi
df_county_list = list(df['county_vaccine'])

def difference(lst):
    final_list = list(set(lst) - set(df_county_list))
    return final_list

svi_diff = difference(svi_county_list)
checkpoint_diff = difference(checkpoint_county_list)
vaccine_diff = difference(vaccine_county_list)
election_diff = difference(election_county_list)
census_diff = difference(census_county_list)

In [25]:
print(f'df svi diff: {len(svi_diff)}')
print(f'df checkpoint diff: {len(checkpoint_diff)}')
print(f'df vaccine diff: {len(vaccine_diff)}')
print(f'df election diff: {len(election_diff)}')
print(f'df census diff: {len(census_diff)}')

df svi diff: 54
df checkpoint diff: 103
df vaccine diff: 54
df election diff: 1539
df census diff: 103


In [26]:
#add 49 empty entries to svi_diff and vaccine_diff so can add everything to a df
vacc_svi_append = ['ZZZZ']*1485
check_census_append = ['ZZZZ']*1436
vaccine_diff = vaccine_diff+vacc_svi_append
svi_diff = svi_diff+vacc_svi_append
checkpoint_diff = checkpoint_diff+check_census_append
census_diff = census_diff+check_census_append

In [27]:
diffs = pd.DataFrame()
diffs['svi_diff ']=svi_diff
diffs['checkpoint_diff']=checkpoint_diff
diffs['vaccine_diff']=vaccine_diff
diffs['election_diff']=election_diff
diffs['census_diff']=census_diff
diffs
diffs.to_csv('./clean_data/diffs.csv')

In [21]:
#0 value counts above 1

#df['County_State'].value_counts(ascending=False)
#checkpoint['County_State'].value_counts(ascending=False)
#svi['County_State'].value_counts(ascending=False)
#vaccine['County_State'].value_counts(ascending=False)
#election['County_State'].value_counts(ascending=False)
#census['County_State'].value_counts(ascending=False)